In [3]:
import spacy
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
scholars = np.load('../gs_scholars_all_upd.npy', allow_pickle=True)

In [ ]:
df_all = pd.DataFrame.from_records(list(scholars))

In [ ]:
df_all.head(2)

,academic,academic_age,academic_lifespan,cit_sum_before_year,citation_table,domain_labels,gender,gs_sid,organization,organization_code,...,co_authors,name,papers,url,top_num_coauthors,num_domain,top_topic_diversity,co_authors_all,num_coauthors_all_wwo_ai,organization_top
0,False,31.0,28.0,"{'1994': 39, '1995': 86, '1996': 187, '1997': ...","[3539, 27]","[logic_programming, machine_learning, natural_...",F,IIrX5SMAAAAJ,N/A,None,...,None,Theresa Swift,[[https://scholar.google.com/citations?view_op...,https://scholar.google.com/citations?hl=en&use...,NaN,3.0,NaN,"{6DDwarUAAAAJ, _6zceo4AAAAJ, vSu9AVYAAAAJ, e23...",84,0.0
1,False,27.0,24.0,"{'1996': 10, '1997': 58, '1998': 203, '1999': ...","[3502, 30]","[natural_language_processing, data_management,...",-,9gVmuMAAAAAJ,"Software Engineering Manager, Apple Inc.",11540454294127154050,...,[[https://scholar.google.com/citations?user=Sr...,George Andrei Mihaila,[[https://scholar.google.com/citations?view_op...,https://scholar.google.com/citations?hl=en&use...,20.0,3.0,9.0,"{DnnCWN0AAAAJ, mQMC3aYAAAAJ, 9jmmp5sAAAAJ, gv2...",115,1.0


generate clusters of organizations

In [ ]:
organizations = []
organization_codes = []
for each in scholars:
  organization = BeautifulSoup(each["extra_info"][0], 'html.parser').text
  try:
    organization_code = each["extra_info"][0].split("org=")[1].split('"')[0]
  except:
    organization_code = None
  organizations.append(organization)
  organization_codes.append(organization_code)
df = pd.DataFrame({'organization': organizations, 'organization_code': organization_codes})

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.researchgate.net/profile/Jeffrey_Howard2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://github.com/puolival" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336:

In [ ]:
df_none = df[df["organization_code"].isnull()]

In [ ]:
df = df.dropna()

In [ ]:
organization2occur = {}
for index, row in df.iterrows():
  if row["organization_code"] not in organization2occur:
    organization2occur[row["organization_code"]] = [row]
  else:
    organization2occur[row["organization_code"]].append(row)

In [ ]:
organization = []
organization_code = []
occurance = []
for each in organization2occur:
  occurance.append(len(organization2occur[each]))
  organization.append(organization2occur[each][0][0])
  organization_code.append(organization2occur[each][0][1])
df = pd.DataFrame({'organization': organization, 'organization_code': organization_code, 'occurance': occurance})

In [ ]:
df = df.sort_values('occurance', ascending=False)
# df_new = df[df["occurance"] > 10]
len(df)

3568

In [ ]:
! pip install spacy
! pip install nltk
! python -m spacy download en_core_web_sm
  
# imports and load spacy english language package
import spacy
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('en_core_web_sm')

In [ ]:
org_ents = []
for index, row in df_none.iterrows():
  text = nlp(row["organization"])
  ents = [(e.text, e.label_) for e in text.ents]
  org_ent = ""
  for each in ents:
    if each[1] == 'ORG':
      org_ent += each[0]
  org_ent = (org_ent if org_ent != "" else None)
  org_ents.append(org_ent)
df_none["org_ent"] = org_ents

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
df_none_new = df_none[df_none["org_ent"].notnull()]

,organization,organization_code,org_ent
14,"Chief Technology Officer, Zensar Technologies ...",None,Zensar Technologies Ltd.
19,Microsoft Turing,None,Microsoft
21,PhD Scholar. University of Science and Technol...,None,University of Science and Technology Bannu
23,Research Scientist at Facebook AI,None,Facebook AI
31,"Principal Data Scientist at Bayer AG, Decision...",None,Bayer AGDecision Science Pharma
...,...,...,...
78040,Professor of Digital Linguistics and Writing R...,None,Digital Linguistics and Writing ResearchZHAW Z...
78049,Meisei University,None,Meisei University
78054,"Postdoc at CSAIL, MIT",None,CSAILMIT
78055,Al-Imam Muhammad Bin Saud Islamic University (...,None,IMSIU


In [ ]:
# Utility function for generating sentence embedding from the text
def get_embeddinngs(text):
    return nlp(text).vector

# Generating sentence embedding from the text
df_none_new['emb'] = df_none_new['organization'].apply(get_embeddinngs)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
! pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import time
org_ent = list(df_none_new["org_ent"].values)
corpus_embeddings = model.encode(org_ent, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=10, threshold=0.75)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Batches:   0%|          | 0/286 [00:00<?, ?it/s]

Start clustering
Clustering done after 1.88 sec


In [ ]:
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", org_ent[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", org_ent[sentence_id])


Cluster 1, #277 Elements 
	 IBM Research
	 IBM Research
	 IBM Research
	 ...
	 IBM Research - Haifa
	 IBM ResearchIndian Institute of Technology DelhiIndian Institute of Technology
	 IBM ResearchHaifa Labs

Cluster 2, #267 Elements 
	 Amazon.com Services LLC
	 Amazon.com Services LLC
	 Amazon.com Services, Inc.
	 ...
	 Amazon
	 Amazon
	 Amazon

Cluster 3, #230 Elements 
	 Microsoft Research
	 Microsoft Research
	 Microsoft Research
	 ...
	 Microsoft Research Redmond
	 Microsoft ResearchUniversity of Washington
	 Microsoft ResearchRedmond

Cluster 4, #212 Elements 
	 Department of Computer Science& EngineeringAmity University
	 Department of Computer ScienceTaif University
	 Department of Computer ScienceUtah State University
	 ...
	 School of Computer Sciences
	 Department of Computer EngineeringMuthiah Polytechnic CollegeAnnamalainagar
	 School of Electronic Engineering and Computer Science

Cluster 5, #159 Elements 
	 University of Chinese
	 Chinese University of Hong Kong
	 Chinese

In [ ]:
df_notnull_cluster = df.sort_values('occurance', ascending=False)
df_new = df_notnull_cluster[df_notnull_cluster["occurance"] > 10]
len(df_new)

927

In [ ]:
for idx, row in df.iterrows():
  if 'mit' in row['organization'].lower():
    print(row['organization'])

MIT, CSAIL
Chief Scientist, MITRE Corporation
IBM CTO for Mitsubishi UFJ Financial Group Americas, STSM, Master Inventor
MIT CSAIL
Summit Health, Mount Sinai
MIT Computer Science and Artificial Intelligence Laboratory
Prometeia, CSAIL at MIT
Dana-Farber Cancer Institute, Broad Institute of MIT & Harvard
MIT CSAIL
Tohoku University, Langsmith inc.
Vice President, Fidelity Investments. Research Affiliate, MIT CSAIL
MIT
Senior Manager, Data Science Capabilities, GlaxoSmithKline
Domino Data Lab, Curata, MIT
Ph.D., Chief Architect at Baidu, Chair of Baidu AI Technical Committee
School of Computing Technologies, RMIT University
Associate Professor, Amity University Haryana
LinkedIn, SkipFlag, MIT
Massachusetts Institute of Technology (MIT)
Outra Limited
Research Scientist at MIT, PhD University of Birmingham
MIT-IBM Watson AI Lab
Postdoc, MIT
Graduate Student, MIT
MIT
MIT
MIT
Principal Scientist, MITRE
NLP/ML Researcher. MIT PhD.
Amity University,  Kolkata
Principal Technology Architect, Inf

In [ ]:
for idx, row in df.iterrows():
  if row['organization'] == 'None':
    # print(row)
    # break
    flag = 0
    str_lower = row['organization'].lower()
    if 'carnegie mellon university' in str_lower:
      flag = 1
    elif 'google' in str_lower:
      flag = 1
    elif 'stanford' in str_lower:
      flag = 1
    elif 'cmu' in str_lower:
      flag = 1
    elif 'mit' in str_lower:
      flag = 1
    elif 'massachusetts institute of technology' in str_lower:
      flag = 1
    elif 'amazon' in str_lower:
      flag = 1
    elif 'aws' in str_lower:
      flag = 1
    elif 'uc berkeley' in str_lower:
      flag = 1
    elif 'university of california, berkeley' in str_lower:
      flag = 1
    elif 'microsoft' in str_lower:
      flag = 1
    elif 'facebook' in str_lower:
      flag = 1
    elif 'ibm' in str_lower:
      flag = 1
    elif 'apple' in str_lower:
      flag = 1
    
    if flag == 1:
      df.at[idx, 'organization_top'] = 1
    else:
      df.at[idx, 'organization_top'] = 0
  else:
    if row['organization_code'] in ['6518679690484165796', 
                                    '8539678734835078480', 
                                    '17554517495610703090', 
                                    '16345133980181568013', 
                                    '11215970455707674271', 
                                    '11816294095661060495', 
                                    '12331530069074480064', 
                                    '1978088699341142706', 
                                    '2122379019098182280', 
                                    '11540454294127154050']:
      df.at[idx, 'organization_top'] = 1
    else:
      df.at[idx, 'organization_top'] = 0

get top 10 organizations
- by manually integrating scholars having organization codes with scholars without organization codes

In [ ]:
pd.set_option('display.max_rows', 500)
df_notnull_cluster.head(100)
# Google 6518679690484165796, Stanford 8539678734835078480, CMU 17554517495610703090, MIT 16345133980181568013, Amazon 11215970455707674271, UCB 11816294095661060495, Microsoft 12331530069074480064, Facebook 1978088699341142706, IBM 2122379019098182280, Apple 11540454294127154050